In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

In [2]:
nielsen15 = pd.read_csv('../../Nielsen/aggregated_nielsen_2015.csv')
nielsen16 = pd.read_csv('../..//Nielsen/aggregated_nielsen_2016.csv')
nielsen15['year'] = 2015
nielsen16['year'] = 2016

In [3]:
nielsen = pd.concat((nielsen15, nielsen16))
nielsen = nielsen[~nielsen.is_walmart]

In [4]:
## Entriy/exit dates
fandom = pd.read_csv('../data_collection/plein_de_data/fandom_traitées.csv', parse_dates=['Opening_date', 'Closing_date'])[['State', 'County_name', 'County_fips', 'Opening_date', 'Closing_date']]

# We drop the state in which we do not trust our data (some mistakes stil remain)
fandom = fandom[~np.isin(fandom.State, ('CA', 'GA', 'KS', 'LA', 'TX'))]

# We concentrate our study on the movements (entries & exits) during the fiscal years 2015 and 2016
movements = fandom[((fandom.Opening_date >= '2015-01-31') & (fandom.Opening_date <= '2017-01-31')) | ((fandom.Closing_date >= '2015-01-31') & (fandom.Closing_date <= '2017-01-31'))]
#movements['year'] = movements.Opening_date.dt.year
#movements['month'] = movements.Opening_date.dt.month

## FIRST MODEL : one month

Regression model :
$$ Price_{i, t} = \alpha + \beta \cdot treat_i + \gamma \cdot post_t + \delta \cdot treat_i \cdot post_{t} + \varepsilon_{i, t}$$

In [5]:
year = 2016
month = 1

In [6]:
# We choose to focus on milk prices
milk = nielsen[nielsen.product_group_descr == 'MILK']


# The control group is composed by all states where nothing (no entry nor exit) happened.
control = milk[~np.isin(milk.guessed_store_county_fips, movements)].copy()
print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")


# The treatment group is composed by the states where one entry took place in august 2016 and where this entry is the only movement
count = movements.groupby('County_fips').count()
count = count[count.Opening_date + count.Closing_date == 1] # No more than one movement in the treatement group
treatment_movements = movements[(np.isin(movements.County_fips, count.index)) & (movements.Opening_date.dt.year == year) & (movements.Opening_date.dt.month == month)]

treatment = milk[np.isin(milk.guessed_store_county_fips, treatment_movements.County_fips )].copy()
print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")


# We create our dummies for the regression
control['treat'] = False
control['post'] = (control.purchase_month > month) & (control.purchase_year == year)
treatment['treat'] = True
treatment['post'] = (treatment.purchase_month > month) & (treatment.purchase_year == year)


# Final dataset for the regression :

df = pd.concat((control, treatment))[['upc_price', 'treat', 'post']]

Size of the control group: 2279.
Size of the treatment group: 15.


In [7]:
reg = smf.ols(formula='upc_price ~ treat * post', data=df)
results = reg.fit()

In [8]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              upc_price   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     160.7
Date:                Wed, 26 Oct 2022   Prob (F-statistic):          1.39e-103
Time:                        14:16:20   Log-Likelihood:                -44825.
No. Observations:               44565   AIC:                         8.966e+04
Df Residuals:                   44561   BIC:                         8.969e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      2.8057      0.004    658.140      0.000       2.797       2.814
treat[T.True]                 -0.1472      0.048     -3.087      0.002      -0.241      -0.054
post[T.True]                  -0.1350      0.006    -21.369      0.000      -0.147      -0.123
treat[T.True]:post[T.True]    -0.0223      0.070     -0.316      0.752      -0.160       0.116
==============================================================================
Omnibus:                    33315.409   Durbin-Watson:                   1.061
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3893044.850
Skew:                           2.830   Prob(JB):                         0.00
Kurtosis:                      48.437   Cond. No.                         28.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## SECOND MODEL : all months

Regression model :
$$ Price_{i, t} = \alpha + \beta \cdot treat_i + \delta \cdot treat_i \cdot post_{t} + \varepsilon_{i, t}$$

In [9]:
# We choose to focus on milk prices
milk = nielsen[nielsen.product_group_descr == 'MILK']


# The control group is composed by all states where nothing (no entry nor exit) happened.
control = milk[~np.isin(milk.guessed_store_county_fips, movements)].copy()
print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")


# The treatment group is composed by the states where one entry took place in 2016 and where this entry is the only movement
count = movements.groupby('County_fips').count()
count = count[count.Opening_date + count.Closing_date == 1] # No more than one movement in the treatement group
treatment_movements = movements[(np.isin(movements.County_fips, count.index))]

treatment = milk[np.isin(milk.guessed_store_county_fips, treatment_movements.County_fips )].copy()
treatment = treatment.merge(treatment_movements, left_on='guessed_store_county_fips', right_on='County_fips')
print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")


# We create our dummies for the regression
control['treat'] = False
control['interaction'] = False
treatment['treat'] = True
treatment['interaction'] = (treatment.purchase_month > treatment.Opening_date.dt.month) & (treatment.purchase_year >= treatment.Opening_date.dt.year)


# Final dataset for the regression :

df = pd.concat((control, treatment))[['upc_price', 'treat', 'interaction']]

Size of the control group: 2279.
Size of the treatment group: 89.


In [10]:
reg = smf.ols(formula='upc_price ~ treat + interaction', data=df)
results = reg.fit()

In [11]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              upc_price   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     9.022
Date:                Wed, 26 Oct 2022   Prob (F-statistic):           0.000121
Time:                        14:16:40   Log-Likelihood:                -46380.
No. Observations:               46322   AIC:                         9.277e+04
Df Residuals:                   46319   BIC:                         9.279e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               2.7443      0.003    876.095      0.000       2.738       2.750
treat[T.True]           0.0261      0.018      1.462      0.144      -0.009       0.061
interaction[T.True]    -0.1244      0.030     -4.108      0.000      -0.184      -0.065
==============================================================================
Omnibus:                    33684.540   Durbin-Watson:                   1.031
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3728294.954
Skew:                           2.718   Prob(JB):                         0.00
Kurtosis:                      46.613   Cond. No.                         10.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## THIRD MODEL : adding time fixed effects

Regression model :
$$ Price_{i, t} = \alpha + \beta \cdot treat_i + \delta \cdot treat_i \cdot post_{t} + \sum_{\tau=Janv15}^{Dec16}\gamma_{\tau} \cdot \mathbb{1}(t=\tau) + \varepsilon_{i, t}$$

In [12]:
# We choose to focus on milk prices
milk = nielsen[nielsen.product_group_descr == 'MILK']


# The control group is composed by all states where nothing (no entry nor exit) happened.
control = milk[~np.isin(milk.guessed_store_county_fips, movements)].copy()
print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")


# The treatment group is composed by the states where one entry took place in 2016 and where this entry is the only movement
count = movements.groupby('County_fips').count()
count = count[count.Opening_date + count.Closing_date == 1] # No more than one movement in the treatement group
treatment_movements = movements[(np.isin(movements.County_fips, count.index))]

treatment = milk[np.isin(milk.guessed_store_county_fips, treatment_movements.County_fips )].copy()
treatment = treatment.merge(treatment_movements, left_on='guessed_store_county_fips', right_on='County_fips')
print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")


# We create our dummies for the regression
control['treat'] = False
control['interaction'] = False
control['time_fixed_effects'] = list(zip(control.purchase_year, control.purchase_month)).strftime('%Y-%m')
treatment['treat'] = True
treatment['interaction'] = (treatment.purchase_month > treatment.Opening_date.dt.month) & (treatment.purchase_year >= treatment.Opening_date.dt.year)
treatment['time_fixed_effects'] = list(zip(treatment.purchase_month, treatment.purchase_year)).strftime('%Y-%m')


# Final dataset for the regression :

df = pd.concat((control, treatment))[['upc_price', 'treat', 'interaction', 'time_fixed_effects']].set_index(['time_fixed_effects'], drop=False)

Size of the control group: 2279.
Size of the treatment group: 89.


AttributeError: 'list' object has no attribute 'strftime'

In [ ]:
import linearmodels as plm

#reg = plm.PanelOLS.from_formula(
    formula='upc_price ~ treat + interaction + TimeEffects',
    data=df,
    drop_absorbed=True)
#results = reg.fit()

In [ ]:
#results.summary()

## THIRD MODEL : adding entity effects

Regression model :
$$ Price_{i, t} = \alpha + \beta \cdot treat_i + \delta \cdot treat_i \cdot post_{t} + \sum_{\tau=Janv15}^{Dec16}\gamma_{\tau} \cdot \mathbb{1}(t=\tau) + \sum_{s \in USStates}\gamma_{s} \cdot \mathbb{1}(state = s) + \varepsilon_{i, t}$$

In [16]:
# We choose to focus on milk prices
milk = nielsen[nielsen.product_group_descr == 'MILK']


# The control group is composed by all states where nothing (no entry nor exit) happened.
control = milk[~np.isin(milk.guessed_store_county_fips, movements)].copy()
print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")


# The treatment group is composed by the states where one entry took place in 2016 and where this entry is the only movement
count = movements.groupby('County_fips').count()
count = count[count.Opening_date + count.Closing_date == 1] # No more than one movement in the treatement group
treatment_movements = movements[(np.isin(movements.County_fips, count.index))]

treatment = milk[np.isin(milk.guessed_store_county_fips, treatment_movements.County_fips )].copy()
treatment = treatment.merge(treatment_movements, left_on='guessed_store_county_fips', right_on='County_fips')
print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")


# We create our dummies for the regression
control['treat'] = False
control['interaction'] = False
control['time_fixed_effects'] = list(zip(control.purchase_month, control.purchase_year))
treatment['treat'] = True
treatment['interaction'] = (treatment.purchase_month > treatment.Opening_date.dt.month) & (treatment.purchase_year >= treatment.Opening_date.dt.year)
treatment['time_fixed_effects'] = list(zip(treatment.purchase_month, treatment.purchase_year))


# Final dataset for the regression :

df = pd.concat((control, treatment))[['upc_price', 'treat', 'interaction', 'time_fixed_effects', 'store_state']]

Size of the control group: 2279.
Size of the treatment group: 89.


In [22]:
reg = smf.ols(formula='np.log(upc_price+1) ~ treat + interaction + C(time_fixed_effects) + C(store_state)', data=df[df.upc_price!=0])
results = reg.fit()

In [28]:
df[df.upc_price < 0.5]

,upc_price,treat,interaction,time_fixed_effects,store_state
14421,0.33,False,False,"(4, 2015)",AL
375747,0.00,False,False,"(2, 2015)",IL
14648,0.33,False,False,"(8, 2016)",AL
144238,0.39,False,False,"(11, 2016)",CO
294778,0.00,False,False,"(12, 2016)",IA
843086,0.00,False,False,"(7, 2016)",ND
843111,0.00,False,False,"(8, 2016)",ND
1188856,0.49,False,False,"(11, 2016)",SD


In [23]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     np.log(upc_price + 1)   R-squared:                       0.176
Model:                               OLS   Adj. R-squared:                  0.174
Method:                    Least Squares   F-statistic:                     135.0
Date:                   Wed, 26 Oct 2022   Prob (F-statistic):               0.00
Time:                           14:41:11   Log-Likelihood:                 21853.
No. Observations:                  46318   AIC:                        -4.356e+04
Df Residuals:                      46244   BIC:                        -4.291e+04
Df Model:                             73                                         
Covariance Type:               nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               1.3868      0.006    235.766      0.000       1.375       1.398
treat[T.True]                          -0.0064      0.004     -1.507      0.132      -0.015       0.002
interaction[T.True]                    -0.0150      0.007     -2.132      0.033      -0.029      -0.001
C(time_fixed_effects)[T.(1, 2016)]     -0.0675      0.005    -13.940      0.000      -0.077      -0.058
C(time_fixed_effects)[T.(2, 2015)]     -0.0208      0.005     -4.314      0.000      -0.030      -0.011
C(time_fixed_effects)[T.(2, 2016)]     -0.0811      0.005    -16.715      0.000      -0.091      -0.072
C(time_fixed_effects)[T.(3, 2015)]     -0.0295      0.005     -6.121      0.000      -0.039      -0.020
C(time_fixed_effects)[T.(3, 2016)]     -0.0915      0.005    -18.881      0.000      -0.101      -0.082
C(time_fixed_effects)[T.(4, 2015)]     -0.0483      0.005     -9.977      0.000      -0.058      -0.039
C(time_fixed_effects)[T.(4, 2016)]     -0.0958      0.005    -19.745      0.000      -0.105      -0.086
C(time_fixed_effects)[T.(5, 2015)]     -0.0494      0.005    -10.201      0.000      -0.059      -0.040
C(time_fixed_effects)[T.(5, 2016)]     -0.0970      0.005    -20.012      0.000      -0.107      -0.088
C(time_fixed_effects)[T.(6, 2015)]     -0.0557      0.005    -11.516      0.000      -0.065      -0.046
C(time_fixed_effects)[T.(6, 2016)]     -0.1059      0.005    -21.841      0.000      -0.115      -0.096
C(time_fixed_effects)[T.(7, 2015)]     -0.0513      0.005    -10.603      0.000      -0.061      -0.042
C(time_fixed_effects)[T.(7, 2016)]     -0.1091      0.005    -22.485      0.000      -0.119      -0.100
C(time_fixed_effects)[T.(8, 2015)]     -0.0498      0.005    -10.322      0.000      -0.059      -0.040
C(time_fixed_effects)[T.(8, 2016)]     -0.1046      0.005    -21.517      0.000      -0.114      -0.095
C(time_fixed_effects)[T.(9, 2015)]     -0.0550      0.005    -11.347      0.000      -0.064      -0.045
C(time_fixed_effects)[T.(9, 2016)]     -0.1038      0.005    -21.355      0.000      -0.113      -0.094
C(time_fixed_effects)[T.(10, 2015)]    -0.0621      0.005    -12.818      0.000      -0.072      -0.053
C(time_fixed_effects)[T.(10, 2016)]    -0.0819      0.005    -16.819      0.000      -0.091      -0.072
C(time_fixed_effects)[T.(11, 2015)]    -0.0552      0.005    -11.378      0.000      -0.065      -0.046
C(time_fixed_effects)[T.(11, 2016)]    -0.0847      0.005    -17.414      0.000      -0.094      -0.075
C(time_fixed_effects)[T.(12, 2015)]    -0.0408      0.005     -8.428      0.000      -0.050      -0.031
C(time_fixed_effects)[T.(12, 2016)]     0.0191      0.005      3.927      0.000       0.010       0.029
C(store_state)[T.AR]              

## THIRD MODEL : adding entity effects

Regression model :
$$
\begin{align}
Price_{i, t} &= \alpha + \beta \cdot treat_i + \sum_{\tau=Feb15}^{Dec16} \delta_{\tau} \cdot treat_i \cdot \mathbb{1}(t=\tau)  + \sum_{\tau=Feb15}^{Dec16}\gamma_{\tau} \cdot \mathbb{1}(t=\tau) + \sum_{s \in USStates}\gamma_{s} \cdot \mathbb{1}(state = s) + \varepsilon_{i, t}\\
&= \beta_i + \sum_{\tau=Feb15}^{Dec16} \delta_{\tau} \cdot treat_i \cdot \mathbb{1}(t=\tau)  + \rho_t + \varepsilon_{i, t}
\end{align}
$$